In [1]:
import numpy as np
from numpy.linalg import*
import matplotlib.pyplot as plt
from numpy import r_, exp, cos, sin, pi, zeros, ones, hanning, sqrt, log, floor, reshape, mean
from scipy import signal
from numpy.fft import fft
import math
%matplotlib inline

In [2]:
# data import
data_path = 'sample_data.csv'
data = np.genfromtxt(data_path, delimiter=',')
Ns = len(data[1:,0])
fs = 100  # [Hz]
dt = 1/fs  # unit time


1770


In [3]:
# Initial orientation calculation
v2_raw = data[0,:]
v2 = v2_raw/np.linalg.norm(v2_raw) 

y=sqrt(1-np.square(v2[1]))
x=-v2[0]*v2[1]/y
z=x*v2[2]/v2[0]

v1 = np.zeros(3)
v1 = [x,y,z]

v0 = np.cross(v1,v2)

X = np.vstack((v0,v1,v2)).T
v0_g = np.array([1,0,0])
v1_g = np.array([0,1,0])
v2_g = np.array([0,0,1])
X_g = np.vstack((v0_g,v1_g,v2_g)).T

R = np.matmul(X_g,np.linalg.inv(X))


[[ 0.99992646  0.         -0.0121273 ]
 [-0.00204041  0.9857445  -0.16823677]
 [ 0.01195441  0.16824915  0.98567201]]
[[ 0.99992646 -0.00204041  0.01195441]
 [ 0.          0.9857445   0.16824915]
 [-0.0121273  -0.16823677  0.98567201]]
[[ 9.99926462e-01 -2.04040710e-03  1.19544148e-02]
 [-4.39952612e-19  9.85744503e-01  1.68249146e-01]
 [-1.21272954e-02 -1.68236773e-01  9.85672013e-01]]
[ 0.99992646 -0.00204041  0.01195441]


In [8]:
# 3D orientation tracking

def getdR(R0,dt,l):
    dth = np.linalg.norm(l,2)*dt    # Rotation Angle
    l_norm = l/np.linalg.norm(l)
    l_g = np.matmul(R0,l_norm.T)
    u_x = np.array([[0,-l_g[2], l_g[1]]
                    ,[l_g[2], 0, -l_g[0]]
                    ,[-l_g[1], l_g[0], 0]])  # cross product matrix
    dR = np.identity(3)*np.cos(dth) + np.sin(dth)*u_x + (1-np.cos(dth))*np.outer(l_g,l_g)
    return dR
    
R_now = R
for i in range(Ns):
    dR = getdR(R_now,dt,data[i+1,:])
    R_now = np.matmul(dR,R_now)


